## 01.오즈와 오즈비 개념 이론


### 01-01. 오즈 (Odds)
- 오즈는 어떤 사건이 발생할 확률과 발생하지 않을 확률의 비율이다.
- 수식: `오즈 = P(사건 발생) / P(사건 미발생)`
- 예: 질병에 걸릴 확률이 0.8이라면 오즈는 0.8 / 0.2 = 4


### 01-02. 오즈비 (Odds Ratio)
- 두 집단 간 오즈의 비율을 의미하며, 사건이 한 집단에서 다른 집단보다 얼마나 자주 발생하는지를 비교한다.
- 수식: `오즈비 = (a / b) / (c / d) = (a*d) / (b*c)`
  - a: 실험군에서 사건 발생 수
  - b: 실험군에서 사건 미발생 수
  - c: 대조군에서 사건 발생 수
  - d: 대조군에서 사건 미발생 수


### 01-03.해석 기준
| OR 값 | 의미 |
|--------|------|
| OR = 1 | 두 그룹의 사건 발생 가능성이 동일함 |
| OR > 1 | 앞 그룹에서 사건 발생 가능성이 더 높음 |
| OR < 1 | 뒤 그룹에서 사건 발생 가능성이 더 높음 |


## 02.약복용여부에 따른 발병여부

### 02-01.예시 상황
| 그룹       | 질병 발생 | 질병 없음 | 전체 |
|------------|------------|------------|-------|
| 약 복용    | 20         | 80         | 100   |
| 비복용     | 40         | 60         | 100   |

- 약 복용 그룹의 오즈: 20 / 80 = 0.25
- 비복용 그룹의 오즈: 40 / 60 = 0.667
- 오즈비: (20 * 60) / (80 * 40) = 1200 / 3200 = 0.375

> 해석: 약 복용 그룹은 비복용 그룹보다 질병 발생 오즈가 더 낮음 → 약이 효과 있음

In [2]:
# # statsmodels 설치 -> 통계 분석 라이브러리
# %pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [4]:
from statsmodels.stats.contingency_tables import Table2x2


# 2x2 교차표: [[약 복용, 비복용],
#             [질병 발생 여부 여부]]
table = [[20, 80],
         [40, 60]]


result = Table2x2(table)
print(f'오즈비: {result.oddsratio}')
print(f'95% 신뢰구간 : {result.oddsratio_confint()}')

오즈비: 0.375
95% 신뢰구간 : (np.float64(0.19920817232354643), np.float64(0.7059198343108242))


## 03. 로지스틱 회귀에서의 오즈비 해석


### 03-01. 예제: 흡연 여부에 따른 질병 발생


**회귀 계수 (coef)** 는 로그 오즈비 (log odds ratio) -> **np.exp(coef)** 는 오즈비


* 오즈비 > 1 → 양의 상관관계 (흡연하면 질병 확률 증가)
* 오즈비 < 1 → 음의 상관관계 (흡연하면 질병 확률 감소)

> 해석: np.exp(coef)가 오즈비. 예: exp(1.2) = 3.32 → 흡연자는 질병 오즈(질병 발생할 비율)가 3.3배


In [8]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression

# 데이터 생성
df = pd.DataFrame({
    'smoker' :[1,1,1,1,0,0,0,0],
    'disease' :[1,1,1,1,0,0,0,1]
})


X = df[['smoker']]
y = df['disease']

model = LogisticRegression()
model.fit(X, y)
print(model.coef_)
coef = model.coef_[0][0]
print(f'회귀 계수 : {coef:.3f}') # == 로그 오즈비
print(f'오즈비 : {np.exp(coef):.3f}' )

[[1.02969183]]
회귀 계수 : 1.030
오즈비 : 2.800


## 04. 범주형 변수에서 오즈비


### 04-01. 교차표 기반 예시
| 성별 | 구매 | 비구매 |
|------|------|--------|
| 남자 | 30   | 70     |
| 여자 | 50   | 50     |


**오즈비가 0.429**
→ 남성은 여성에 비해 구매할 오즈가 더 낮다 (약 0.43배)


**95% 신뢰구간이 (0.229, 0.805)**
→ 이 구간에 1이 포함되지 않음(1이 포함되면, 오즈비가 모호하다는 것. 1보다 작거나, 커야한다)
→ 통계적으로 유의미하게 여성의 구매 오즈가 높다고 말할 수 있음


결론
- 여성은 남성보다 구매할 가능성이 높다.
- 오즈 기준으로 보면 약 2.3배 (1 / 0.429 ≈ 2.33) 더 많이 구매함
- 신뢰구간도 1보다 작으므로 유의미한 차이임


In [10]:
table = [[30, 70],
         [50, 50]]


result = Table2x2(table)
print(f'오즈비: {result.oddsratio:.3f}')
print(f'95% 신뢰구간 : {result.oddsratio_confint()}')
# 남자는 여자보다 구매할 오즈가 0.43 더 낮다


오즈비: 0.429
95% 신뢰구간 : (np.float64(0.23991818978964746), np.float64(0.7655670858003472))


# 04-02. 로지스틱 회귀로 범주형 변수 오즈비 확인

In [13]:
df = pd.DataFrame({
    'gender': ['male']*100 + ['female']*100,
    'buy': [1]*30 + [0]*70 + [1]*50 + [0]*50
})

df['gender_code'] = df['gender'].map({'male' : 0, 'female' : 1}) # 인코딩

X = df[['gender_code']]
y = df['buy']

model = LogisticRegression()
model.fit(X, y)


coef = model.coef_[0][0]
print (f'오즈비(여성/남성) : {np.exp(coef):.3f}') # 지수복원

# 여자의 구매 비율이 남자의 구매 비율보다 2.18 높다.
# 확률이 아니고 오즈가 몇 배다 라는 것을 의미

오즈비(여성/남성) : 2.180


## 05. 다범주형 범주 변수의 오즈비


### 05-01. 예제: 직업별 구매 여부


> 해석:
> - engineer의 오즈비: teacher 대비 구매 가능성이 몇 배인지
> - student의 오즈비: teacher 대비 낮은지 높은지 판단

In [19]:
df = pd.DataFrame({
    'job': ['teacher', 'engineer', 'student', 'teacher', 'student', 'engineer',
            'teacher', 'student', 'engineer', 'student'] * 10,
    'buy': [1, 1, 0, 1, 0, 1, 0, 1, 0, 0] * 10
})

# 더미 변수화
# get_dummies : 범주형 변수를 0/1 숫자 컬럼들로 쪼개는 함수
df_encoded = pd.get_dummies(df, columns=['job']).astype(int)
# display(df_encoded)

X = df_encoded[['job_engineer', 'job_student']]
y = df_encoded['buy']

model= LogisticRegression()
model.fit(X, y)

coef = model.coef_[0]
odds_ratios = np.exp(coef)


print('회귀계수 :', coef)
print('오즈비 :', odds_ratios)


회귀계수 : [ 0.16034906 -1.42550824]
오즈비 : [1.17392056 0.24038626]
